In [215]:
import pandas as pd
import torch

In [216]:
data = pd.read_csv("seeds.csv")

In [217]:
data

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
194,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
195,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
196,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
197,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [218]:
data.columns

Index(['Area', 'Perimeter', 'Compactness', 'Kernel.Length', 'Kernel.Width',
       'Asymmetry.Coeff', 'Kernel.Groove', 'Type'],
      dtype='object')

In [219]:
data["Type"].unique()

array([1, 2, 3], dtype=int64)

In [220]:
# data["island"].unique()

In [221]:
data.isnull().sum()

Area               0
Perimeter          0
Compactness        0
Kernel.Length      0
Kernel.Width       0
Asymmetry.Coeff    0
Kernel.Groove      0
Type               0
dtype: int64

In [222]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='most_frequent')
# data.iloc[:,:] = imputer.fit_transform(data)

In [223]:
# data.isnull().sum()

In [224]:
# from sklearn.preprocessing import LabelEncoder
# lb = LabelEncoder()
# data["sex"] = lb.fit_transform(data["sex"])
# data["island"] = lb.fit_transform(data["island"])

In [225]:
data["Type"].value_counts()

2    68
1    66
3    65
Name: Type, dtype: int64

In [226]:
data

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
194,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
195,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
196,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
197,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [227]:
# data.drop("rowid",axis=1)

In [228]:
X = torch.tensor(data.drop("Type", axis=1).values, dtype=torch.float)
y = torch.tensor(
    data["Type"], 
    dtype=torch.long
)

In [229]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [230]:
X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32),torch.tensor(X_test, dtype=torch.float32),torch.tensor(Y_train, dtype=torch.float32),torch.tensor(Y_test, dtype=torch.float32)
samples, features = X_train.shape

C:\Users\Bharath D\AppData\Local\Temp\ipykernel_19436\4043573011.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32),torch.tensor(X_test, dtype=torch.float32),torch.tensor(Y_train, dtype=torch.float32),torch.tensor(Y_test, dtype=torch.float32)


In [231]:
from torch import nn

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.first_layer = nn.Linear(features, 3)
        self.second_layer = nn.Linear(3, 10)
        self.third_layer = nn.Linear(10, 15)
        self.fourth_layer = nn.Linear(15, 30)
        self.fifth_layer = nn.Linear(30, 35)
        self.final_layer = nn.Linear(35,1)
        self.relu = nn.ReLU()

    def forward(self, X_batch):
        layer_out = self.relu(self.first_layer(X_batch))
        layer_out = self.relu(self.second_layer(layer_out))
        layer_out = self.relu(self.third_layer(layer_out))
        layer_out = self.relu(self.fourth_layer(layer_out))
        layer_out = self.relu(self.fifth_layer(layer_out))

        return self.final_layer(layer_out)

In [232]:
regressor = Regressor()
preds = regressor(X_train[:5])
preds

tensor([[0.1105],
        [0.1031],
        [0.0949],
        [0.1061],
        [0.1127]], grad_fn=<AddmmBackward0>)

In [233]:
def TrainModel(model, loss_func, optimizer, X, Y, epochs=500):
    for i in range(epochs):
        preds = model(X) ## Make Predictions by forward pass through network

        loss = loss_func(preds.ravel(), Y) ## Calculate Loss

        optimizer.zero_grad() ## Zero weights before calculating gradients
        loss.backward() ## Calculate Gradients
        optimizer.step() ## Update Weights

        if i % 100 == 0: ## Print MSE every 100 epochs
            print("MSE : {:.2f}".format(loss))

In [234]:
from torch.optim import SGD, RMSprop, Adam

torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 1000
learning_rate = torch.tensor(1/1e3) # 0.001

regressor = Regressor()
mse_loss = nn.MSELoss()
optimizer = SGD(params=regressor.parameters(), lr=learning_rate)

TrainModel(regressor, mse_loss, optimizer, X_train, Y_train, epochs=epochs)

MSE : 4.29
MSE : 0.78
MSE : 0.77
MSE : 0.76
MSE : 0.75
MSE : 0.74
MSE : 0.73
MSE : 0.71
MSE : 0.70
MSE : 0.68


In [235]:
test_preds = regressor(X_test) ## Make Predictions on test dataset
test_preds[:5]

tensor([[1.9931],
        [1.9950],
        [2.1881],
        [1.9368],
        [2.1870]], grad_fn=<SliceBackward0>)

In [236]:
train_preds = regressor(X_train) ## Make Predictions on train dataset
train_preds[:5]

tensor([[1.8003],
        [1.8051],
        [1.8144],
        [1.8388],
        [1.9005]], grad_fn=<SliceBackward0>)

In [237]:
from sklearn.metrics import r2_score
print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy().squeeze(), Y_train.detach().numpy())))
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy().squeeze(), Y_test.detach().numpy())))

Train R^2 Score : -25.28
Test  R^2 Score : -21.90


In [238]:
# test_preds = regressor(X_test) ## Make Predictions on test dataset

# test_preds = torch.argmax(test_preds, axis=1) ## Convert Probabilities to class type

# train_preds = regressor(X_train) ## Make Predictions on train dataset

# train_preds = torch.argmax(train_preds, axis=1) ## Convert Probabilities to class type

# test_preds[:5], train_preds[:5]

(tensor([0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0]))

In [239]:
# from sklearn.metrics import accuracy_score

# print("Train Accuracy : {:.2f}".format(accuracy_score(Y_train, train_preds)))
# print("Test  Accuracy : {:.2f}".format(accuracy_score(Y_test, test_preds)))

Train Accuracy : 0.00
Test  Accuracy : 0.00


In [240]:
# from sklearn.metrics import classification_report

# print("Test Data Classification Report : ")
# print(classification_report(Y_test, test_preds))

Test Data Classification Report : 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00       9.0
         2.0       0.00      0.00      0.00      19.0
         3.0       0.00      0.00      0.00      12.0

    accuracy                           0.00      40.0
   macro avg       0.00      0.00      0.00      40.0
weighted avg       0.00      0.00      0.00      40.0



C:\Users\Bharath D\.conda\envs\env-2\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bharath D\.conda\envs\env-2\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bharath D\.conda\envs\env-2\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bharath D\.conda\e

In [214]:
y

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3])